In [3]:
from andrew_utils import MTurkManager
import json
import pandas as pd
import glob
import numpy as np
from datetime import datetime

In [4]:
def parse_timestring(s):
    return datetime.strptime(s, '%Y-%m-%dT%H:%M:%S.%fZ')

In [5]:
data_directory = '/Users/andrew/Desktop/add_experiment/data/exp01'

raw_data = {}
for filename in glob.glob(data_directory + '/*'):
    with open(filename) as f:
        data = json.load(f)
        worker_id = data['hit_params']['workerId']
        if worker_id == 'testWorkerId':
            continue
        raw_data[worker_id] = data
        data['start'] = parse_timestring(data['start'])
        data['end'] = parse_timestring(data['end'])
        data['duration'] = (data['end'] - data['start']).seconds

In [6]:
compensations = []
for worker_id in raw_data:
    data = raw_data[worker_id]
    hit_id = data['hit_params']['hitId']
    assignment_id = data['hit_params']['assignmentId']
    base_compensation = .05
    correct_reward = data['app_settings']['correctReward']
    incorrect_penalty = data['app_settings']['incorrectPenalty']
    correct = data['summary']['total_correct']
    incorrect = data['summary']['total_incorrect']
    bonus = correct*correct_reward - incorrect*incorrect_penalty - base_compensation
    extra_bonus = 0
    total = bonus + extra_bonus + base_compensation
    compensations.append({
        'worker_id': worker_id,
        'approve': correct > incorrect,
        'base_compensation': base_compensation,
        'hit_id': hit_id,
        'assignment_id': assignment_id,
        'bonus': max(0, bonus),
        'extra_bonus': max(0, extra_bonus),
        'message': 'You solved {} problems and missed {} problems.'.format(correct, incorrect),
        'duration': data['duration'],
        'wage': total / (data['duration']/3600)
    })
compensation_df = pd.DataFrame(compensations)

In [7]:
print(compensation_df[compensation_df.approve].wage.sum())
compensation_df[compensation_df.approve].wage.describe()

114.72142876545195


count    10.000000
mean     11.472143
std       3.385962
min       3.067938
25%      10.957794
50%      12.712581
75%      13.514352
max      14.021598
Name: wage, dtype: float64

In [8]:
compensation_df

,worker_id,approve,base_compensation,hit_id,assignment_id,bonus,extra_bonus,message,duration,wage
0,A11FFJZ0C984W4,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3YDTZAI2WZ4R89OYFEM6XRFNOUR41B,5.36,0,You solved 575 problems and missed 34 problems.,1394,13.971306
1,A2HHGI5CNSS5PD,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3BV8HQ2ZZYP1HK5CUL8E5QS4OFZ6AI,5.10,0,You solved 562 problems and missed 47 problems.,1375,13.483636
2,A35ROC8J91VOW0,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,34V1S5K3GUPNC1V72FUE9VHKIXH693,5.03,0,You solved 557 problems and missed 49 problems.,2095,8.729356
3,A24DSECWUBZC0Z,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3TVSS0C0E3O4QVJRR3CZYPH906CTWQ,4.03,0,You solved 508 problems and missed 100 problems.,1377,10.666667
4,A2F0A4FIVT9M3O,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3TPZPLC3M20975OW02SDOI5FDAY3PQ,1.38,0,You solved 363 problems and missed 220 problems.,1678,3.067938
5,A2GOYSTIL3LOV1,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3X4MXAO0BICRAG6ZNE74YAGRCLFRWB,5.36,0,You solved 575 problems and missed 34 problems.,1389,14.021598
6,AA6RYE4V0IBKM,True,0.05,3QE4DGPGBSZLQ4G77V27FUN435U4GU,3BWI6RSP7IXNBX0TT0FS7NIR3XNE79,5.79,0,You solved 596 problems and missed 12 problems.,1777,11.831176
7,A34DR0CVUBDL1N,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3SUWZRL0M0147XMOAQA019IV9UX6ED,5.39,0,You solved 576 problems and missed 32 problems.,1506,13.003984
8,A1GRPIBHW72HDU,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3KWTYT0872RMUZW2MTNZMYUFMWN5LJ,5.75,0,You solved 593 problems and missed 13 problems.,1681,12.421178
9,A3C0NQ2IP5FTZQ,True,0.05,3RIHDBQ1NFMFUQ0OX5KLMOG54YSHM5,3Y54SXRO1N9RYRUMW3XYG1WCFQ1TUR,5.45,0,You solved 579 problems and missed 29 problems.,1464,13.524590


In [6]:
mturk = MTurkManager(compensation_df,
                     '/Users/andrew/Documents/mturk_keys/AWS_ACCESS_KEY_ID',
                     '/Users/andrew/Documents/mturk_keys/AWS_SECRET_ACCESS_KEY')
mturk.total_bonus_payout

1 workers found in local data but not in MTurk servers. They will be removed.: 
A2FGP5K2VWW23G


In [7]:
responses = mturk.approve_all()
responses

{}

In [8]:
responses = mturk.reject_all()
responses

{}

In [9]:
mturk.compensation_df[mturk.compensation_df.approve].bonus.sum()*1.2

116.64

In [10]:
responses = mturk.bonus_all()

In [11]:
responses

{'A1EOPEG6RHXO1V': True,
 'A1J7YOEECGUE2L': True,
 'A1KFTQRBMXSC13': True,
 'A1PNYLOKED8FWF': True,
 'A1WELSASLGM0P4': True,
 'A226PPE8EN1W8M': True,
 'A27A3Z6J7745DI': True,
 'A2871R3LEPWMMK': True,
 'A2EVTBJG1Z9TCP': True,
 'A2KHMKJUVDMK1N': True,
 'A2O2Q0LVWOWJ1G': True,
 'A2O359OPIQNMDL': True,
 'A2RCMJLGDWYFO': True,
 'A2WY6HLF2YLSOF': True,
 'A315GZMPR49WIY': True,
 'A37DKV7XQI7N5J': True,
 'A3LE1BIVORW3VR': True,
 'A83OOIGU4606R': True,
 'A8R445009SICT': True,
 'AC5FF0SJI0RXM': True,
 'AC7QPNOH2LELF': True,
 'ALHS0TDN03O9K': True,
 'AMONQ9MU8EJ2': True}

In [12]:
mturk.compensation_df

,assignment_id,approve,hit_id,worker_id,base_compensation,bonus,extra_bonus,message,assignment_status
0,3QXNC7EIPKJSYQ9S0QEX0U7T0XB09D,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,AC5FF0SJI0RXM,0.05,4.15,0,You solved 481 problems and missed 61 problems.,Approved
1,3S06PH7KSTS4857R8PNZ0V9TWAWD1A,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A1WELSASLGM0P4,0.05,3.86,0,You solved 469 problems and missed 78 problems.,Approved
2,3JPSL1DZ5UN9O5B4T2UV71EQE84AND,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A37DKV7XQI7N5J,0.05,4.56,0,You solved 505 problems and missed 44 problems.,Approved
3,3XXU1SWE8OJ639JXSWSXWPID6FT0AS,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A315GZMPR49WIY,0.05,3.43,0,You solved 443 problems and missed 95 problems.,Approved
4,324G5B4FB5WOK7LZLTGDMYGN57N071,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A2RCMJLGDWYFO,0.05,3.78,0,You solved 465 problems and missed 82 problems.,Approved
5,3X87C8JFV8ZYRYAQQU83WV315PXSQD,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A1KFTQRBMXSC13,0.05,4.48,0,You solved 499 problems and missed 46 problems.,Approved
6,3PWWM24LHUMZGVTWALCXJM5XDPD82G,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A3LE1BIVORW3VR,0.05,4.94,0,You solved 524 problems and missed 25 problems.,Approved
7,33TIN5LC06YPVL5D8YP6SM1DCAPY9B,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A2WY6HLF2YLSOF,0.05,4.90,0,You solved 521 problems and missed 26 problems.,Approved
8,3BF51CHDTXYLDWRBY22PDB5HJAJ0H3,True,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A2871R3LEPWMMK,0.05,3.77,0,You solved 463 problems and missed 81 problems.,Approved
9,37QW5D2ZRIASLUAU40R3A4I1BJC8SJ,False,367O8HRHKHWOG9PY2VWO4867QZ9S4O,A1NWXL4QO30M8U,0.05,0.00,0,You solved 109 problems and missed 436 problems.,Rejected
